## Librerias

In [81]:

# Tratamiento de datos
import numpy as np
import pandas as pd
import re

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go

# Estadísticas
import scipy.stats as stats
from scipy.stats import f_oneway
from scipy.stats import shapiro

# modelos de aprendizaje supervisado 
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
import pickle
 
# mis cosas
import sys
import os

import urllib.request
from PIL import Image

_________________________________
INICIO BASELINE

In [ ]:
def limp_pre_fe (df)


FIN BASELINE
_________________________________

## Data

Primero voy a probar con mi df limpio, y probar todos los metodos del modelo excepto preprocess pues ya lo tengo hecho. Una vez compruebe que funciona, puedo incorporar todo ese paquete de transformaciones en este metodo de Skmodel y ver si funciona también.

In [82]:
### Cargamos los datos procesados y limpios en el notebook anterior
df = pd.read_parquet ('data/processed/com_proceso_2.parquet')

In [83]:
df

,Company,2 in 1 Convertible,Gaming,Netbook,Notebook,Ultrabook,Workstation,"Inches("")",Touchscreen,IPS,Pixels,CPU Maker,CPU(Ghz),RAM (GB),Memory SSD (GB),Memory HDD (GB),Memory SSHD (GB),GPU Maker,Weight (kg),Price_euros
laptop_ID,,,,,,,,,,,,,,,,,,,,
599,1,1,0,0,0,0,0,11.6,1,0,1049088,0,1.0,4,128,0,0,0,1.39,603.00
92,2,0,1,0,0,0,0,15.6,0,0,2073600,1,3.0,8,0,1024,0,1,2.45,699.00
744,3,0,0,0,0,0,1,15.6,0,0,2073600,0,2.8,16,256,0,0,2,2.60,2419.00
729,3,1,0,0,0,0,0,13.3,1,1,1049088,0,2.5,8,256,0,0,0,1.58,836.60
957,4,0,0,0,1,0,0,15.6,0,0,1049088,0,2.3,8,256,0,0,0,2.20,1119.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,9,0,0,1,0,0,0,11.6,0,1,1049088,0,1.6,4,32,0,0,0,1.35,355.00
626,9,1,0,0,0,0,0,11.6,1,1,1049088,0,1.6,4,32,0,0,0,1.25,379.00
225,1,0,0,0,1,0,0,15.6,0,0,2073600,0,1.6,8,256,0,0,1,2.20,677.35


In [84]:
import os
import pickle
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

class Skmodel:
    def __init__(self, model_name, model_type, model_version):
        self.model_name = model_name
        self.model_type = model_type
        self.model_version = model_version
        
        if model_type == 'Regresión Lineal':
            self.model = LinearRegression()
        elif model_type == 'Regresión Logística':
            self.model = LogisticRegression()
        else: 
            raise ValueError("Tipo de modelo no soportado: elija entre 'Regresión Lineal' o 'Regresión Logística'")

    def split(self, df):
        """Divide el dataset en entrenamiento y prueba."""
        target_column = 'Price_euros'  # Nombre de la variable objetivo
        test_size = 0.2  # Porcentaje de prueba
        random_state = 42  # Semilla para reproducibilidad
        
        X = df.drop(columns=[target_column])
        y = df[target_column]
        
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state
        )
        print(f"Datos divididos: {len(self.X_train)} entrenamiento, {len(self.X_test)} prueba")

    def preprocess(self, df):
        """Preprocesa los datos seleccionando las columnas relevantes."""
        columns = ['RAM (GB)', 'Memory SSD (GB)', 'Memory HDD (GB)', 'Memory SSHD (GB)', 'Price_euros']
        df = df[columns]
        return df  # Devuelve el DataFrame procesado

    def train(self):
        """Entrena el modelo con los datos divididos previamente."""
        if not hasattr(self, 'X_train'):
            raise ValueError("Debes ejecutar 'split()' antes de entrenar el modelo.")
        
        self.model.fit(self.X_train, self.y_train)
        print("Modelo entrenado correctamente.")

    def predict(self, input_data):
        """Genera predicciones a partir del modelo entrenado."""
        return self.model.predict(input_data)

    def evaluate(self):
        """Evalúa el modelo con los datos de prueba."""
        if not hasattr(self, 'X_test'):
            raise ValueError("Debes ejecutar 'split()' antes de evaluar el modelo.")
        
        score = self.model.score(self.X_test, self.y_test)
        print(f"Model Score: {score:.4f}")
        return score

    def save(self, output_path=None):
        """Guarda el modelo entrenado en un archivo pickle."""
        if output_path is None:
            output_path = f'data/results/{self.model_name}_v{self.model_version}.pkl'
        
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        with open(output_path, 'wb') as f:
            pickle.dump(self.model, f)
        
        print(f"Modelo guardado en {output_path}")


In [74]:
class Skmodel:
    def __init__(self, model_name, model_type, model_version):
        self.model_name = model_name
        self.model_type = model_type
        self.model_version = model_version
        if model_type == 'Regresión Lineal':
            self.model = LinearRegression()
        elif model_type == 'Regresión Logística':
            self.model = LogisticRegression()
        else: 
            raise ValueError("Tipo de modelo no soportado: elija entre 'Regresión Lineal' o 'Regresión Logística'")
        pass
    
    def split(self, df):
        target_column = 'Price_euros'  # Define el nombre de la columna objetivo
        test_size = 0.2  # Define el tamaño del conjunto de prueba
        random_state = 42  # Define la semilla aleatoria
        X = df.drop(columns=[target_column])
        y = df[target_column]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split (X, y, test_size = test_size, random_state = random_state)

    def preprocess(self, df):
        # coloco aquí un ejemplo de lo que haría 
        df = df[['RAM (GB)','Memory SSD (GB)','Memory HDD (GB)','Memory SSHD (GB)','Price_euros']]
        
    
    def train(self, train_data, test_data):
        train_data = X_train, y_train
        test_data = X_test, y_test 
        self.model.fit(X_train, y_train)
        self.X_test = X_test
        self.y_test = y_test

    def predict(self, input_data):
        return self.model.predict(input_data)

    def evaluate(self, test_data):
        X_test, y_test = test_data
        score = self.model.score(X_test, y_test)
        print(f"Model Score: {score}")
        return score
    
    def save(self, output_path):
        # crea el directorio si no existe. Si existe, nada.
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        output_path = 'data/results/example_model{model_version}'
        with open(output_path, 'wb') as f:
            pickle.dump(self.model, f)
        print(f"Model saved to {output_path}")
    

In [92]:
model = Skmodel ('ordenadores', 'Regresión Lineal', '1.0')



In [93]:
model.split(df)

Datos divididos: 729 entrenamiento, 183 prueba


In [94]:
model.preprocess(df)

,RAM (GB),Memory SSD (GB),Memory HDD (GB),Memory SSHD (GB),Price_euros
laptop_ID,,,,,
599,4,128,0,0,603.00
92,8,0,1024,0,699.00
744,16,256,0,0,2419.00
729,8,256,0,0,836.60
957,8,256,0,0,1119.00
...,...,...,...,...,...
837,4,32,0,0,355.00
626,4,32,0,0,379.00
225,8,256,0,0,677.35


In [95]:
model.train()

Modelo entrenado correctamente.


In [97]:
model.evaluate()

Model Score: 0.7977


0.7977131219682133

In [98]:
model.save()

Modelo guardado en data/results/ordenadores_v1.0.pkl


## Data exploration

In [ ]:
train.info()

In [ ]:
train.head()

## Data processing

In [5]:
train = train[['laptop_ID','Inches','Price_euros']]

In [ ]:
train

In [7]:
# df.to_csv("Dataset/limpio.csv")

-----------------------------------------------------------------------------------------------------------------

## Pasos que si o si debéis realizar para poder participar en la competición:

### 1. Definir X e y

In [ ]:
X = train.drop(['laptop_ID','Price_euros'], axis=1)
y = train['Price_euros'].copy()
X.shape

In [ ]:
y.shape

### 2. Dividir X_train, X_test, y_train, y_test

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
X_train

### 3. Asignar el modelo (vacío) a una variable


In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()

regressor.fit(X_train, y_train)

### 4. Sacar métricas, valorar el modelo 

Recuerda que en la competición se va a evaluar con la métrica de RMSE.

In [13]:
from sklearn.metrics import root_mean_squared_error

predictions = regressor.predict(X_test)

rmse = root_mean_squared_error(y_test, predictions)

In [ ]:
print(rmse)

-----------------------------------------------------------------

## Una vez listo el modelo, toca predecir con el dataset de predicción 

Definición de **modelo que está listo**. 

Tras hacer suficientes pruebas, analizar los datos, hacer feature engineering, probar diferentes modelos con diferentes parámetros, es con este con el que observo mejores métricas y menos overfitting. ¡Cuidado con el overfitting aquí! Si vuestro modelo aprende muy bien de estos datos pero hay overfitting cuando le pasemos los datos desconocidos de `test.csv` nos arriesgamos a que digamos, no salga lo esperado.

### 1. Entrena dicho modelo con TODOS tus datos de train, esto es con `train.csv` al completo.


**CON LAS TRANSFORMACIONES QUE LE HAYAS REALIZADO A `X` INCLUÍDAS.**


Véase:
- Estandarización/Normalización
- Eliminación de Outliers
- Eliminación de columnas
- Creación de columnas nuevas
- Gestión de valores nulos
- Y un largo etcétera de técnicas que como Data Scientist hayas considerado las mejores para tu dataset.

### 2. Carga los datos de `test.csv` para predecir.

In [ ]:
X_pred = pd.read_csv("data/com_test.csv")
X_pred.head()

In [ ]:
X_pred.shape

**¿Por qué da error?** 

In [ ]:
predictions_submit = regressor.predict(X_pred)
predictions_submit

IMPORTANTE: APLICAR LO MISMO A ESTOS DATOS QUE HAYÁIS APLICADO A LOS DATOS DE ENTRENAMIENTO

- SI EL ARRAY CON EL QUE HICISTEIS `.fit()` ERA DE 4 COLUMNAS, PARA `.predict()` DEBEN SER LAS MISMAS
- SI AL ARRAY CON EL QUE HICISTEIS `.fit()` LO NORMALIZASTEIS, PARA `.predict()` DEBÉIS NORMALIZARLO
- TODO IGUAL SALVO BORRAR FILAS, EL NÚMERO DE ROWS SE DEBE MANTENER EN ESTE SET, PUES LA PREDICCIÓN DEBE TENER 391 FILAS, SI O SI

**Entonces, si al cargar los datos de train usé `index_col=0` para que utilizara la primera columna del conjunto de datos como índice, ¿tendré que hacerlo también para el conjunto `test.csv`?**

In [ ]:
# ¿Qué opináis?
# Sí

![](https://www.tierraljelechu.com/web/img/wiki_up/1.996-SorpresaDto.-1-Red.jpg)

In [ ]:
X_pred = pd.read_csv("data/com_test.csv")
X_pred.head()

In [ ]:
X_pred = X_pred[['Inches']]
X_pred

### 3. AHORA puedo hacer la predicción que será lo que subirás a Kaggle. 

**¿Qué es lo que subirás a Kaggle?**

In [ ]:
predictions_submit = regressor.predict(X_pred)
predictions_submit

**¡PERO! Para subir a Kaggle la predicción, ésta tendrá que tener una forma específica y no valdrá otra.**

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [22]:
sample = pd.read_csv("data/com_sample_submission.csv")

In [ ]:
sample.head()

In [ ]:
sample.shape

### 4. Mete tus predicciones en un dataframe. 

En este caso, la **MISMA** forma que `sample_submission.csv`. 

In [25]:
submission = pd.DataFrame({"laptop_ID": sample['laptop_ID'], "Price_euros": predictions_submit})

In [ ]:
submission.head()

In [ ]:
submission.shape

### 5. Pásale el CHEQUEATOR para comprobar que efectivamente está listo para subir a Kaggle.

In [28]:
def chequeator(df_to_submit,version):
    """
    Esta función se asegura de que tu submission tenga la forma requerida por Kaggle.
    
    Si es así, se guardará el dataframe en un `csv` y estará listo para subir a Kaggle.
    
    Si no, LEE EL MENSAJE Y HAZLE CASO.
    
    Si aún no:
    - apaga tu ordenador, 
    - date una vuelta, 
    - enciendelo otra vez, 
    - abre este notebook y 
    - leelo todo de nuevo. 
    Todos nos merecemos una segunda oportunidad. También tú.
    """
    if df_to_submit.shape == sample.shape:
        if df_to_submit.columns.all() == sample.columns.all():
            if df_to_submit['laptop_ID'].all() == sample['laptop_ID'].all():
                print("You're ready to submit!")
                submission.to_csv(f"submission_{version}.csv", index = False) #muy importante el index = False
                urllib.request.urlretrieve("https://i.kym-cdn.com/photos/images/facebook/000/747/556/27a.jpg", "gfg.png")     
                img = Image.open("gfg.png")
                img.show()   
            else:
                print("Check the ids and try again")
        else:
            print("Check the names of the columns and try again")
    else:
        print("Check the number of rows and/or columns and try again")
        print("\nMensaje secreto de Clara: No me puedo creer que después de todo este notebook hayas hecho algún cambio en las filas de `diamonds_test.csv`. Lloro.")


In [ ]:
chequeator(submission,1)